### Install dependency

In [1]:
!pip install onnx_tf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 11.0 MB/s eta 0:00:00


In [2]:
import os
import sys
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import onnx
import onnx_tf
import tensorflow as tf

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
models_path = "/dataset/working/"
onnx_model_path = os.path.join(models_path, 'model.onnx')
tf_model_path = os.path.join(models_path, 'tf_model')
tflite_model_path = os.path.join(models_path, 'model.tflite')
inference_args_path = os.path.join(models_path, 'inference_args.json')


In [4]:
class Model(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(Model, self).__init__()
        self.linear = nn.Linear(in_dim, out_dim)

    def forward(self, h):
        h = torch.tensor(h)
        h[torch.isnan(h)] = 0
       
        # print(h.shape)
        h = self.linear(h)
        return h


In [5]:
selected_columns = ['x_right_hand_0', 'y_right_hand_0', 'z_right_hand_0',
                    'x_right_hand_1', 'y_right_hand_1', 'z_right_hand_1',
                    'x_right_hand_2', 'y_right_hand_2', 'z_right_hand_2',
                    'x_right_hand_3', 'y_right_hand_3', 'z_right_hand_3',
                    'x_right_hand_4', 'y_right_hand_4', 'z_right_hand_4',
                    'x_right_hand_5', 'y_right_hand_5', 'z_right_hand_5',
                    'x_right_hand_6', 'y_right_hand_6', 'z_right_hand_6',
                    'x_right_hand_7', 'y_right_hand_7', 'z_right_hand_7',
                    'x_right_hand_8', 'y_right_hand_8', 'z_right_hand_8',
                    'x_right_hand_9', 'y_right_hand_9', 'z_right_hand_9',
                    'x_right_hand_10', 'y_right_hand_10', 'z_right_hand_10',
                    'x_right_hand_11', 'y_right_hand_11', 'z_right_hand_11',
                    'x_right_hand_12', 'y_right_hand_12', 'z_right_hand_12',
                    'x_right_hand_13', 'y_right_hand_13', 'z_right_hand_13',
                    'x_right_hand_14', 'y_right_hand_14', 'z_right_hand_14',
                    'x_right_hand_15', 'y_right_hand_15', 'z_right_hand_15',
                    'x_right_hand_16', 'y_right_hand_16', 'z_right_hand_16',
                    'x_right_hand_17', 'y_right_hand_17', 'z_right_hand_17',
                    'x_right_hand_18', 'y_right_hand_18', 'z_right_hand_18',
                    'x_right_hand_19', 'y_right_hand_19', 'z_right_hand_19',
                    'x_right_hand_20', 'y_right_hand_20', 'z_right_hand_20',
                    ]

selected_columns_dict = {"selected_columns": selected_columns}

with open(inference_args_path, "w") as f:
    json.dump(selected_columns_dict, f)



In [6]:
n_class = 59
opset_version = 12
NUM_FEATURES = len(selected_columns)

pq_path = "/dataset/input/asl-fingerspelling/train_landmarks/1019715464.parquet"

sample_input = pd.read_parquet(pq_path, columns=selected_columns)
sample_input = sample_input.values
print(sample_input.shape)

(161722, 63)


In [7]:
print("converting from torch to onnx...")

model = Model(in_dim = NUM_FEATURES,
              out_dim = n_class)

sample_input = torch.tensor(sample_input)
torch.onnx.export(
    model,                       # PyTorch Model
    sample_input,                # Input tensor
    onnx_model_path,             # Output file (eg. 'output_model.onnx')
    opset_version=opset_version, # Operator support version
    input_names=['inputs'],      # Input tensor name (arbitary)
    output_names=['outputs'],    # Output tensor name (arbitary)
    dynamic_axes={
        'inputs': {0: 'length',
                   },
        'outputs': {0: 'length',
                    },

    }
)

converting from torch to onnx...


In [8]:
print("converting from onnx to tensorflow...")
onnx_model = onnx.load(onnx_model_path)
tf_model = onnx_tf.backend.prepare(onnx_model)
tf_model.export_graph(tf_model_path)

converting from onnx to tensorflow...


In [9]:
print("converting from tensorflow to tflite")
converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_path)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
# converter.target_spec.supported_types = [tf.float32]
converter.experimental_new_converter = True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
                                       tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()
# Save the model
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)
print("converting finish!!")
# %%

converting from tensorflow to tflite
converting finish!!


In [10]:
!zip submission.zip $tflite_model_path $inference_args_path


  adding: dataset/working/model.tflite (deflated 18%)
  adding: dataset/working/inference_args.json (deflated 84%)
